# Standardizing a Database

prefixes for all of the support groups  
what ing or trn do they have access to?  
What service-account(s) do they have?  
what access does the service account have?

Making it work for all of the environ:



In [2]:
from environments import ENVIRONMENTS
from applications import APPLICATIONS, Application
from service_role import ServiceRole

In [3]:
DEFAULT_SCHEMAS = ['_ing', '_trn']


## Applications

In [5]:
sql = {}

# ENVIRONMENTS
# Every env has applications
for env in ENVIRONMENTS:
    sql[env] = []
    environment = ENVIRONMENTS.get(env)
    
    # APPLICATIONS
    # Every application has service roles, prefix, group_name, support group
    for application in APPLICATIONS:
        app = Application(application, environment=environment)

        # SUPPORT GROUPS
        support_group_sql = app.create_support_group_sql
        sql[env].append(support_group_sql)
      
        # SERVICE ROLES
        # # Every service_role has env prefix, group name, service_role_name
        for service_role in app.service_roles:
            role = ServiceRole(application_name=app.application_name, service_role=service_role, environment=environment)
            service_account_sql = [
                role.create_group_sql,
                role.create_service_user_sql,
                role.alter_default_privileges_service_account,
                role.grant_usage_to_service_account_ing,
                role.grant_create_to_service_account_ing,
                role.grant_usage_to_service_account_trn,
                role.grant_create_to_service_account_trn,
            ]
            sql[env].extend(service_account_sql)


In [6]:
sql['prod']

['create group "tbdp-core-prod-supp";',
 'create group "tbdp-core-integrate";',
 'create user "s-tbdp-core-integrate-p@toyota.com" password \'DthWTs=6*(#S\';',
 'alter default privileges for user "s-tbdp-core-integrate-p@toyota.com" grant select on tables to group "tbdp-core-integrate";',
 'grant usage on schema "core_ing" to group "tbdp-core-integrate"',
 'grant create on schema "core_ing" to group "tbdp-core-integrate"',
 'grant usage on schema "core_trn" to group "tbdp-core-integrate"',
 'grant create on schema "core_trn" to group "tbdp-core-integrate"',
 'create group "tbdp-core-tableau-egress";',
 'create user "s-tbdp-core-tableau-egress-p@toyota.com" password \'n8TCPWTN)iig\';',
 'alter default privileges for user "s-tbdp-core-tableau-egress-p@toyota.com" grant select on tables to group "tbdp-core-tableau-egress";',
 'grant usage on schema "core_ing" to group "tbdp-core-tableau-egress"',
 'grant create on schema "core_ing" to group "tbdp-core-tableau-egress"',
 'grant usage on sc